In [0]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import KFold
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt

#Import the right one! Don't do 'from keras.models import Sequential' . Need the 'tensorflow' in front!
#STOP USING KERAS! JUST USE tensorflow.keras

In [0]:
height = 28
width = 28
num_classes = 10
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [3]:
#Reshape into a 3D matrix from (height,width) to (height,width,1) . *1 here because its black and white. Set to 3 if its color
'''
X_train = np.reshape(X_train,(X_train.shape[0],height,width,1))
X_test = np.reshape(X_test,(X_test.shape[0],height,width,1))
'''

'\nX_train = np.reshape(X_train,(X_train.shape[0],height,width,1))\nX_test = np.reshape(X_test,(X_test.shape[0],height,width,1))\n'

In [4]:
#One Hot Encode the labels
'''
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
'''

'\ny_train = to_categorical(y_train, num_classes=10)\ny_test = to_categorical(y_test, num_classes=10)\n'

In [0]:
#Get the hold out data
def load_data():
  fashion_mnist = keras.datasets.fashion_mnist
  (X_train, y_train) , (X_test,y_test) = fashion_mnist.load_data()
  X_train = np.reshape(X_train,(X_train.shape[0],height,width,1))
  X_test = np.reshape(X_test,(X_test.shape[0],height,width,1))
  y_train = to_categorical(y_train, num_classes=10)
  y_test = to_categorical(y_test, num_classes=10)
  return X_train,y_train , X_test,y_test


In [6]:
#Preprocess training data

#Convert to int32 type
'''
X_train = X_train.astype('int32')
X_test = X_test.astype('int32')
'''

#Normalize (To a value between 0 and 255, since max value is 255)\
'''
X_train = X_train/255.0
X_test = X_test/255.0
'''

'\nX_train = X_train/255.0\nX_test = X_test/255.0\n'

In [0]:
#Preprocess pixel data
def preprocess_pixel(X_train, X_test):
  X_train = X_train.astype('int32')
  X_test = X_test.astype('int32')
  X_train = X_train/255.0
  X_test = X_test/255.0
  return X_train , X_test

In [0]:
def define_model():
  #Define the layers
  model = Sequential()
  model.add(Conv2D(64, padding='same',kernel_size=(3,3), activation='relu',kernel_initializer='he_uniform',input_shape=(28,28,1)))  #Kernel_initializer is what initializes the weights at first
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu',kernel_initializer='he_uniform'))
  model.add(Dense(num_classes, activation='softmax'))


  #Compile the model
  opt = SGD(lr=0.01, momentum=0.9)
  model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
  return model

In [0]:
#Note that this function takes X_train and y_train as input. It will then split the data into X_train and X_val (These are LOCAL variables!!). 
def evaluate_model(X,y,n_folds=5):
  scores = []
  histories = []
  #Use K fold cross validation. Basically, split data into 5 sections. Use 1 section as test to evaluate on and the other 4 sections as training. Repeat this for 5 times: So, all data (5 sections) will be used as test and training
  kfold = KFold(n_folds,shuffle=True,random_state=1)  #Shuffle the datast before splitting into 5 sections. Since random_state is used, the 'random' order will be same all the time. The 5 sections for all 5 iterations will be the same
  for train_data_idx , val_data_idx in kfold.split(X):   #Get the INDEX of the splitted train (4 sections) and test (1 section) data using the .split() method for kfold
    model = define_model()
    #Using the indexes obtained from .split() , get the train and test data
    X_train, y_train = X[train_data_idx] , y[train_data_idx]
    X_val, y_val = X[val_data_idx] , y[val_data_idx]
    #Fit the model. History is a tensorflow callback that contains the values 'loss' , 'val_loss' , 'accuracy' and 'val_accuracy' for one whole round (Consist of 10 epochs) (These values are per epoch, so total no of values for the callback for one round is 10*4 = 40  because there are 10 epochs and 4 values given per round)
    history = model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_val,y_val),verbose=1)
    #evaluate model
    _ , acc = model.evaluate(X_test,y_test,verbose=1)  #Returns loss and accuracy, but I only need accuracy here. This tests 
    print(f'Accuracy is :{acc*100}')
    histories.append(history)
    scores.append(acc)
  return scores , histories  

In [0]:
#Plot the learning curve during training session (How the loss and accuracy varies with time)
def summary_learning_curve(histories):
  for i in range(len(histories)):
    #Plot the Loss
    plt.subplot(2,1,1)  #1st Part of a 2 by 1 picture   (rows,columns,part no.)
    plt.title('Cross Entropy Loss')
    plt.plot(histories[i].history['loss'], color='blue', label ='train')  #loss is the loss of the test set. val_loss is the loss of the validation set
    plt.plot(histories[i].history['val_loss'], color='orange',label='test')
    #Plot accuracy
    plt.subplot(2,1,2) #2nd part of the 2 by 1 picture
    plt.title('Classification accuracy')
    plt.plot(histories[i].history['accuracy'], color='blue', label='train')
    plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
  plt.show()

In [0]:
def summary_performance(scores):
  print(f'Accuracy: mean={mean(scores)*100}, SD={std(scores)*100} and n={len(scores)}')
  plt.boxplot(scores)
  plt.show()

In [12]:
#This will give you a more indepth understanding of how well your model is performing. For the real thing, you dont need the summary_learning_curve, summary_performance and evaluate_model. 
#For the real thing , you just need to define model and fit on all of X_train and Y_train. Then test it on X_test and y_test

#This will run for 5 times (Because KFold validation with n_folds=5, so 5 times in total). You will have total of 50 epochs (But its 5 individual sets of 10 epochs each, with each set having a different training and validation set)
'''
X_train,y_train,X_test,y_test = preprocess_data(X_train, y_train, X_test, y_test)
X_train , X_test = preprocess_pixel(X_train,X_test)
scores , histories = evaluate_model(X_train,y_train)
summary_learning_curve(histories)
summary_performance(scores)
'''

'\nX_train,y_train,X_test,y_test = preprocess_data(X_train, y_train, X_test, y_test)\nX_train , X_test = preprocess_pixel(X_train,X_test)\nscores , histories = evaluate_model(X_train,y_train)\nsummary_learning_curve(histories)\nsummary_performance(scores)\n'

In [13]:
#This is extracting the training accuracy from the 5th round. It returns a list of 10 values (As there are 10 epochs and at the end of one epoch the accuracy is outputted once) 
'''
print(histories[4].history['accuracy'])
'''

"\nprint(histories[4].history['accuracy'])\n"

In [14]:
#Since KFold runs 5 times, scores will have 5 values. The scores is based on the evaluation of the model on X_val and y_val, which were subsets of X_test and y_test 
#After every round, there will be one accuracy value (Basically the accuracy of the last epoch of that round). This will be appended to the 'scores' list
'''
print(scores)
'''

'\nprint(scores)\n'

In [0]:
from tensorflow.keras.models import load_model

In [16]:
#This is the main run
X_train,y_train,X_test,y_test = load_data()
X_train , X_test = preprocess_pixel(X_train,X_test)
model = define_model()
model.fit(X_train,y_train,epochs=10, batch_size=32,verbose=1)
model.save('final_model.h5')

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4349 - accuracy: 0.8440
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2837 - accuracy: 0.8979
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2404 - accuracy: 0.9128
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2138 - accuracy: 0.9220
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1902 - accuracy: 0.9306
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1754 - accuracy: 0.9352
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1590 - accuracy: 0.9424
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1431 - accuracy: 0.9470
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1316 - accuracy: 0.9511
Epoch 10/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.119

In [26]:
#Load the weights of the model, that was obtained (saved) after training on the X_train data set
model = load_model('final_model.h5')
_ , acc = model.evaluate(X_test, y_test,verbose=1)
print(f'The accuracy is {acc*100}')

313/313 [==============================] - 1s 2ms/step - loss: 0.2848 - accuracy: 0.9126
The accuracy is 91.25999808311462


In [0]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [0]:
def load_image(filename):
  img = load_img(filename,color_mode='grayscale',target_size=(28,28)) #Convert to grayscale, make to desired shape
  img = img_to_array(img)
  #Reshape to correct dimensions
  img = np.reshape(img, (1,28,28,1))
  #Format the pixels
  img = img.astype('int32')
  img = img/255.0
  #Return the preprocessed image that can be predicted on
  return img

In [0]:
def predict_img():
  img = load_image('pullover.jpg')
  model = load_model('final_model.h5')
  result = model.predict_classes(img)
  print(result[0])

In [40]:
predict_img()

8
